In [1]:
# !pip install transformers[sentencepiece]
# !pip install datasets
# !pip install accelerate -U
# !pip install evaluate
# !pip install pyvi
# !pip install torch

In [1]:
# Các hằng số/siêu tham số

PHOBERT_CHECKPOINT = r"vinai/phobert-base-v2"
INPUT_MAPPING = {'negative': 0, 'neutral': 1, 'positive': 2}
OUTPUT_MAPPING = {'LABEL_0': 'negative', 'LABEL_1': 'neutral', 'LABEL_2': 'positive'}

NUMS_LABEL = 3
SEED = 42
SENTIMENT_ANALYSIS_CHECKPOINT = r"..\python\models\trained_bert\phoBert_v2_Trainer_30k"
SPECIALIZED_STOCK_CHECKPOINT = r"..\python\models\trained_bert\phoBert_v2_specialized"
DECISION_THRESHOLD = 0.69

# Support functions

In [4]:
import torch  # type: ignore # noqa: F401
import re
import pandas as pd

In [5]:
from pyvi import ViTokenizer

def text_preprocessing(text):
    """Tiền xử lý input cho phoBert tokenizer: Loại bỏ các ký tự trắng thừa và ghép các từ tiếng_Việt

    Args:
        text (_type_): _description_

    Returns:
        _type_: _description_
    """    

    text = re.sub(r'\s+', ' ', text).strip()
    text = ViTokenizer.tokenize(text)

    return text

def preprocess_data(data):
    """Tiền xử lý dữ liệu đầu vào

    Args:
        data (DataFrame): DataFrame với 2 cột" 'input' và 'label', giá trị của label phải tuân theo INPUT_MAPPING

    Returns:
        DataFrame: DataFrame đã được tiền xử lý
    """    
    
    preprocessed_data = pd.DataFrame()
    preprocessed_data['input'] = data['input'].apply(text_preprocessing)
    preprocessed_data['label'] = data['label'].replace(INPUT_MAPPING)
    return preprocessed_data

In [6]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import RobertaForSequenceClassification

def prepare_raw_data(df, seed = SEED):
    """Xử lý dữ liệu đầu vào.

    Args:
        data (DataFrame): DataFrame đã được tiền xử lý với 2 cột" 'input' và 'label'
        seed (int, optional): Để có thể thực hiện lại được. Defaults to SEED.

    Returns:
        list: datasets đã được chia train/validation/test (theo 70/15/15) và dataset chưa được chia
    """    
    raw_data = Dataset.from_pandas(df)
    dataset_dict = raw_data.train_test_split(test_size=0.3, shuffle=True, seed=seed)

    # Access the train and test splits
    train_data = dataset_dict["train"]
    test_data = dataset_dict["test"]

    # Split the test_data into eval and test sets
    test_data_dict = test_data.train_test_split(test_size=0.5, shuffle=True, seed=seed)

    # Access the train and eval splits
    eval_data = test_data_dict["train"]
    test_data = test_data_dict["test"]

    # Create a DatasetDict object and combine the datasets
    raw_datasets = DatasetDict({"train": train_data, "validation": eval_data, "test": test_data})
    
    return raw_datasets, raw_data

def prepare_for_trainer(df, checkpoint, seed = SEED):
    """Chuẩn bị sẵn sằng cho quá trình training

    Args:
        data (DataFrame): DataFrame đã được tiền xử lý với 2 cột" 'input' và 'label'
        checkpoint (str): Tên hoặc đường dẫn tới pre-trained model
        seed (int, optional): Để có thể thực hiện lại được. Defaults to SEED.

    Returns:
        list: Gồm model, tokenizer, data_collator, datasets đã được tokenize, datasets ban đầu đã được chia train/validation/test (theo 70/15/15) và dataset chưa được chia
    """

    raw_datasets, raw_data = prepare_raw_data(df)
    phobert = RobertaForSequenceClassification.from_pretrained(checkpoint, num_labels=NUMS_LABEL)

    CHECKPOINT = r"vinai/phobert-base-v2" 
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
    
    def tokenize_function(dataset):
        """Tokenize input

        Args:
            dataset (dataset): dataset dữ liệu đầu vào đã qua tiền xử lý

        Returns:
            dataset: dataset dữ liệu đầu vào đã qua xử lý (tokenize)
        """    
        return tokenizer(dataset["input"], padding = True,  truncation=True, max_length = 256)
    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding= True, max_length=256)

    return phobert, tokenizer, data_collator, tokenized_datasets, raw_datasets, raw_data

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def decision_threshold(row):
    """Để tránh sai sót trong chủ đề nhạy cảm, model phân tích cảm xúc sẽ chỉ đưa ra nhận định tốt/xấu 
    nếu kết quả vượt trên DECISION_THRESHOLD

    Args:
        row (row): row trong DataFrame

    Returns:
        str: Kết quả cuối cùng của phân tích cảm xúc
    """    
    if row['score'] > DECISION_THRESHOLD:
        return row['label']
    else:
        return 'neutral'

def process_results(pipeline_results):
    """Xử lý kết quả đầu ra của model

    Args:
        pipeline_results (_type_): Đầu ra của pipeline phân tích cảm xúc

    Returns:
        DataFrame: DataFrame đã qua xử lý chuyển đổi label, đưa ra quyết định cuối cùng
    """  

    results_df = pd.DataFrame(pipeline_results)
    results_df['label'] = results_df['label'].replace(OUTPUT_MAPPING)
    results_df['final_label'] = results_df.apply(decision_threshold, axis=1)
    return results_df

def evaluate_model(pipeline_results, labels):
    """Đánh giá model trong quá trình test

    Args:
        pipeline_results (_type_): Đầu ra của pipeline phân tích cảm xúc
        labels (iterable): Giá trị kiểm tra 

    Returns:
        dict: Gồm các kết quả của chỉ tiêu accuracy, precision, recall, f1, và confusion matrix
    """    
    results_df = process_results(pipeline_results)
    predictions = results_df["label"]
    y_pred = predictions.to_numpy()
    
    labels = pd.Series(labels)
    labels = labels.replace({v:k for k,v in INPUT_MAPPING.items()})
    y_test = labels.to_numpy()
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    
    return {"accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "confusion_matrix": cm
            }

In [8]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Inferrence

## Preprocessing input data

In [41]:
# Nhập và tiền xử lý dữ liệu

input_path = r"..\python\inputs\df_stock_1804.xlsx"

data = pd.read_excel(input_path)
data

,url,lastmod,stockCode,content,sentiment
0,https://cafef.vn/pvoil-len-tieng-vu-he-thong-b...,2024-04-03T17:21:38+07:00,PVO,"PVOIL bị tấn công bất hợp pháp, hệ thống công ...",negative
1,https://cafef.vn/vinpearl-bao-lai-nam-thu-2-li...,2024-04-03T19:22:19+07:00,VPL,Vinpearl công bố thông tin tình hình tài chính...,positive
2,https://cafef.vn/vinpearl-bao-lai-nam-thu-2-li...,2024-04-03T19:22:19+07:00,VIC,Vingroup tách Vinpearl và thành lập công ty co...,positive
3,https://cafef.vn/cong-ty-mai-tang-sap-tra-co-t...,2024-04-03T15:36:53+07:00,CPH,CTCP Phục vụ Mai táng Hải Phòng (MCK: CPH) thô...,neutral
4,https://cafef.vn/loi-nhuan-doanh-nghiep-boc-ho...,2024-04-03T15:34:10+07:00,LTG,Công ty CP Tập đoàn Lộc Trời (mã chứng khoán: ...,negative
...,...,...,...,...,...
782,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,CEO,Ông Kim Jong Seok – CEO Chứng khoán NH Việt Na...,positive
783,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,NHS,Chứng khoán NH Việt Nam là thành viên thuộc tậ...,positive
784,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,2023,"Năm 2023, NHSV ghi nhận kết quả kinh doanh tíc...",positive
785,https://cafef.vn/chuyen-dong-moi-tu-lien-doanh...,2024-03-26T08:27:58+07:00,YP2C,Khởi công dự án Industrial Centre YP2C thuộc K...,neutral


In [42]:
# Loại bỏ những dữ liệu đầu vào quá dài
max_length = 500
data = data[data['content'].str.len() <= max_length]
data.reset_index(inplace=True)
data

,index,url,lastmod,stockCode,content,sentiment
0,0,https://cafef.vn/pvoil-len-tieng-vu-he-thong-b...,2024-04-03T17:21:38+07:00,PVO,"PVOIL bị tấn công bất hợp pháp, hệ thống công ...",negative
1,1,https://cafef.vn/vinpearl-bao-lai-nam-thu-2-li...,2024-04-03T19:22:19+07:00,VPL,Vinpearl công bố thông tin tình hình tài chính...,positive
2,2,https://cafef.vn/vinpearl-bao-lai-nam-thu-2-li...,2024-04-03T19:22:19+07:00,VIC,Vingroup tách Vinpearl và thành lập công ty co...,positive
3,3,https://cafef.vn/cong-ty-mai-tang-sap-tra-co-t...,2024-04-03T15:36:53+07:00,CPH,CTCP Phục vụ Mai táng Hải Phòng (MCK: CPH) thô...,neutral
4,4,https://cafef.vn/loi-nhuan-doanh-nghiep-boc-ho...,2024-04-03T15:34:10+07:00,LTG,Công ty CP Tập đoàn Lộc Trời (mã chứng khoán: ...,negative
...,...,...,...,...,...,...
777,782,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,CEO,Ông Kim Jong Seok – CEO Chứng khoán NH Việt Na...,positive
778,783,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,NHS,Chứng khoán NH Việt Nam là thành viên thuộc tậ...,positive
779,784,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,2023,"Năm 2023, NHSV ghi nhận kết quả kinh doanh tíc...",positive
780,785,https://cafef.vn/chuyen-dong-moi-tu-lien-doanh...,2024-03-26T08:27:58+07:00,YP2C,Khởi công dự án Industrial Centre YP2C thuộc K...,neutral


In [43]:
# Tiền xử lý
input_data = pd.DataFrame()
input_data["input"] = data["content"].apply(text_preprocessing)
input_data

,input
0,"PVOIL bị tấn_công bất_hợp_pháp , hệ_thống công..."
1,Vinpearl công_bố thông_tin tình_hình tài_chính...
2,Vingroup tách Vinpearl và thành_lập công_ty_co...
3,CTCP Phục_vụ Mai_táng Hải_Phòng ( MCK : CPH ) ...
4,Công_ty CP Tập_đoàn Lộc_Trời ( mã chứng_khoán ...
...,...
777,Ông Kim_Jong_Seok – CEO Chứng_khoán NH Việt_Na...
778,Chứng_khoán NH Việt_Nam là thành_viên thuộc tậ...
779,"Năm 2023 , NHSV ghi_nhận kết_quả kinh_doanh tí..."
780,Khởi_công dự_án Industrial Centre YP2C thuộc K...


In [44]:
_, raw_dataset = prepare_raw_data(input_data)
raw_dataset["input"]

['PVOIL bị tấn_công bất_hợp_pháp , hệ_thống công_nghệ thông_tin bị ngưng_trệ , PVOIL đã điều_tra nguyên_nhân và khắc_phục sự_cố , dự_kiến sẽ đưa các ứng_dụng hoạt_động trở_lại trong 1 đến 2 ngày tới .',
 'Vinpearl công_bố thông_tin tình_hình tài_chính năm 2023 , lãi 670,6 tỷ đồng và tăng vốn chủ sở_hữu lên 13.316 tỷ đồng .',
 'Vingroup tách Vinpearl và thành_lập công_ty_con mới , chuẩn_bị niêm_yết trong năm 2024 .',
 'CTCP Phục_vụ Mai_táng Hải_Phòng ( MCK : CPH ) thông_báo chốt quyền trả cổ_tức 2023 bằng tiền , cổ_đông sở_hữu 1 cổ_phiếu được nhận 1.848 đồng , thời_gian thực_hiện vào 26 / 4 , lãi sau thuế_giảm 12 % , kế_hoạch trả cổ_tức giảm về 16,4 %',
 'Công_ty CP Tập_đoàn Lộc_Trời ( mã chứng_khoán : LTG ) công_bố báo_cáo tài_chính hợp_nhất kiểm_toán năm 2023 , với lãi_ròng giảm từ 265 tỷ đồng trước kiểm_toán xuống còn gần 17 tỷ đồng , nghĩa_là “ bốc_hơi ” tới 248 tỷ đồng - tương_đương mức giảm 94 % .',
 'Công_ty CP City_Auto ( mã chứng_khoán : CTF ) còn 44 tỷ đồng , giảm khoảng 2 tỷ 

## Inferrence

In [45]:
# Tạo pipeline để vận hành model

from transformers import pipeline

sentiment_classifier = pipeline("sentiment-analysis", model = SPECIALIZED_STOCK_CHECKPOINT, tokenizer = PHOBERT_CHECKPOINT)

In [46]:
results = sentiment_classifier(raw_dataset['input'])

KeyboardInterrupt: 

In [47]:
results_df = process_results(results)
results_df

,label,score,final_label
0,neutral,0.844796,neutral
1,positive,0.988668,positive
2,positive,0.767627,positive
3,neutral,0.852302,neutral
4,negative,0.979825,negative
...,...,...,...
777,positive,0.982977,positive
778,positive,0.977807,positive
779,positive,0.993127,positive
780,positive,0.882904,positive


In [48]:
data['sentiment'] = results_df['final_label']
data

C:\Users\Admin\AppData\Local\Temp\ipykernel_11452\1798591180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'] = results_df['final_label']


,index,url,lastmod,stockCode,content,sentiment
0,0,https://cafef.vn/pvoil-len-tieng-vu-he-thong-b...,2024-04-03T17:21:38+07:00,PVO,"PVOIL bị tấn công bất hợp pháp, hệ thống công ...",neutral
1,1,https://cafef.vn/vinpearl-bao-lai-nam-thu-2-li...,2024-04-03T19:22:19+07:00,VPL,Vinpearl công bố thông tin tình hình tài chính...,positive
2,2,https://cafef.vn/vinpearl-bao-lai-nam-thu-2-li...,2024-04-03T19:22:19+07:00,VIC,Vingroup tách Vinpearl và thành lập công ty co...,positive
3,3,https://cafef.vn/cong-ty-mai-tang-sap-tra-co-t...,2024-04-03T15:36:53+07:00,CPH,CTCP Phục vụ Mai táng Hải Phòng (MCK: CPH) thô...,neutral
4,4,https://cafef.vn/loi-nhuan-doanh-nghiep-boc-ho...,2024-04-03T15:34:10+07:00,LTG,Công ty CP Tập đoàn Lộc Trời (mã chứng khoán: ...,negative
...,...,...,...,...,...,...
777,782,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,CEO,Ông Kim Jong Seok – CEO Chứng khoán NH Việt Na...,positive
778,783,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,NHS,Chứng khoán NH Việt Nam là thành viên thuộc tậ...,positive
779,784,https://cafef.vn/chung-khoan-nh-viet-nam-khai-...,2024-03-26T07:14:15+07:00,2023,"Năm 2023, NHSV ghi nhận kết quả kinh doanh tíc...",positive
780,785,https://cafef.vn/chuyen-dong-moi-tu-lien-doanh...,2024-03-26T08:27:58+07:00,YP2C,Khởi công dự án Industrial Centre YP2C thuộc K...,positive
